In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import math
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
iris_data = load_iris() 
x = iris_data.data
y_ = iris_data.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)

#print(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

/home/grumptitan/.local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [3]:
def relu(x):
    x[x<0]=0
    return x
def softmax(arr):
#     arr = arr/np.max(arr)
    return np.exp(arr)/(np.sum(np.exp(arr),axis=0))
def diff_relu(arr):
    z = np.zeros(arr.shape)
    z[arr<=0] = 0
    z[arr>0] = 1
    return z

In [4]:
def initializer(fan_out,fan_in):
    limit = np.sqrt(2*1.0/(fan_in+fan_out))
#     return np.random.uniform(-limit,limit,(fan_out,fan_in))
    return np.random.normal(0,limit,(fan_out,fan_in))

In [5]:
### architecture ###
in_dim = 4
hid1_dim = 10
hid2_dim = 10
out_dim = 3
W1 = initializer(hid1_dim,in_dim)
b1 = initializer(hid1_dim,1)
W2 = initializer(hid2_dim,hid1_dim)
b2 = initializer(hid2_dim,1)
W3 = initializer(out_dim,hid2_dim)
b3 = initializer(out_dim,1)

In [6]:
epochs = 500
num_samples = len(train_x)
batch_size = 1
delta_const = 1e-10
num_batches = num_samples/batch_size
eps = 1e-3
r = {"W1" : np.zeros(W1.shape) , "W2" : np.zeros(W2.shape) ,"W3" : np.zeros(W3.shape),"b1": np.zeros(b1.shape)
     ,"b2" :np.zeros(b2.shape) , "b3" : np.zeros(b3.shape)}
for i in range(epochs):
    (x_train_subs,y_train_subs) = shuffle(train_x,train_y,random_state = 40)
    loss = 0
    for j in range(int(num_batches)):
        W1_upd = np.zeros((hid1_dim,in_dim))
        b1_upd = np.zeros((hid1_dim,1))
        W2_upd = np.zeros((hid2_dim,hid1_dim))
        b2_upd = np.zeros((hid2_dim,1))
        W3_upd = np.zeros((out_dim,hid2_dim))
        b3_upd = np.zeros((out_dim,1))
        for k in range(batch_size):
            z1 = relu(np.matmul(W1,x_train_subs[j]).reshape(-1,1)+b1)
        
            z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)

            out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)
        
            loss = loss + -np.log(out[np.argmax(y_train_subs[j])])
        
            del_3 = out - y_train_subs[j].reshape(-1,1)
            del_2 = np.matmul(W3.T,del_3)*diff_relu(z2)
            del_1 = np.matmul(W2.T,del_2)*diff_relu(z1)

            b3_upd += del_3
#         b3_upd = b3_upd.reshape(len(b3),1)
            b2_upd += del_2
#         b2_upd = b2_upd.reshape(len(b2),1)
            b1_upd += del_1
#         b1_upd = b1_upd.reshape(len(b1),1)
            W3_upd += np.matmul(del_3,z2.T)
            W2_upd += np.matmul(del_2,z1.T)
            W1_upd += np.matmul(del_1,x_train_subs[j].reshape(-1,1).T)
        r["W1"] = r["W1"] + W1_upd*W1_upd
        r["W2"] = r["W2"] + W2_upd*W2_upd
        r["W3"] = r["W3"] + W3_upd*W3_upd
        r["b1"] = r["b1"] + b1_upd*b1_upd
        r["b2"] = r["b2"] + b2_upd*b2_upd
        r["b3"] = r["b3"] + b3_upd*b3_upd
        W3 = W3 - (eps*W3_upd)/(delta_const + np.sqrt(r["W3"]))
        W2 = W2 - (eps*W2_upd)/(delta_const + np.sqrt(r["W2"]))
        W1 = W1 - (eps*W1_upd)/(delta_const + np.sqrt(r["W1"]))
        b3 = b3 - (eps*b3_upd)/(delta_const + np.sqrt(r["b3"]))
        b2 = b2 - (eps*b2_upd)/(delta_const + np.sqrt(r["b2"]))
        b1 = b1 - (eps*b1_upd)/(delta_const + np.sqrt(r["b1"]))
        
    print("Epoch: " + str(i) + " Loss: " + str(loss/num_samples))

Epoch: 0 Loss: [2.04852162]
Epoch: 1 Loss: [1.92808324]
Epoch: 2 Loss: [1.85572404]
Epoch: 3 Loss: [1.79959588]
Epoch: 4 Loss: [1.75094407]
Epoch: 5 Loss: [1.70230433]
Epoch: 6 Loss: [1.65010259]
Epoch: 7 Loss: [1.60193355]
Epoch: 8 Loss: [1.55711668]
Epoch: 9 Loss: [1.5146096]
Epoch: 10 Loss: [1.47490392]
Epoch: 11 Loss: [1.43900804]
Epoch: 12 Loss: [1.40699622]
Epoch: 13 Loss: [1.37753109]
Epoch: 14 Loss: [1.34956381]
Epoch: 15 Loss: [1.32274108]
Epoch: 16 Loss: [1.29692056]
Epoch: 17 Loss: [1.27197607]
Epoch: 18 Loss: [1.24789286]
Epoch: 19 Loss: [1.22457575]
Epoch: 20 Loss: [1.20204797]
Epoch: 21 Loss: [1.18019338]
Epoch: 22 Loss: [1.15901819]
Epoch: 23 Loss: [1.13850775]
Epoch: 24 Loss: [1.11868594]
Epoch: 25 Loss: [1.0997071]
Epoch: 26 Loss: [1.08146543]
Epoch: 27 Loss: [1.06382312]
Epoch: 28 Loss: [1.04677637]
Epoch: 29 Loss: [1.03036659]
Epoch: 30 Loss: [1.01446571]
Epoch: 31 Loss: [0.99905998]
Epoch: 32 Loss: [0.98413401]
Epoch: 33 Loss: [0.96962556]
Epoch: 34 Loss: [0.9555402

Epoch: 290 Loss: [0.35957884]
Epoch: 291 Loss: [0.35928312]
Epoch: 292 Loss: [0.35898235]
Epoch: 293 Loss: [0.35868549]
Epoch: 294 Loss: [0.3583657]
Epoch: 295 Loss: [0.35805072]
Epoch: 296 Loss: [0.35774613]
Epoch: 297 Loss: [0.35744353]
Epoch: 298 Loss: [0.35713733]
Epoch: 299 Loss: [0.35683177]
Epoch: 300 Loss: [0.35652616]
Epoch: 301 Loss: [0.356226]
Epoch: 302 Loss: [0.35592631]
Epoch: 303 Loss: [0.35562772]
Epoch: 304 Loss: [0.35531819]
Epoch: 305 Loss: [0.35501375]
Epoch: 306 Loss: [0.35470752]
Epoch: 307 Loss: [0.35440159]
Epoch: 308 Loss: [0.3540969]
Epoch: 309 Loss: [0.35379864]
Epoch: 310 Loss: [0.35349532]
Epoch: 311 Loss: [0.35319516]
Epoch: 312 Loss: [0.35289086]
Epoch: 313 Loss: [0.35258486]
Epoch: 314 Loss: [0.35227917]
Epoch: 315 Loss: [0.35197425]
Epoch: 316 Loss: [0.35166768]
Epoch: 317 Loss: [0.35135952]
Epoch: 318 Loss: [0.35105113]
Epoch: 319 Loss: [0.35074469]
Epoch: 320 Loss: [0.35043354]
Epoch: 321 Loss: [0.35012177]
Epoch: 322 Loss: [0.3498065]
Epoch: 323 Loss

In [7]:
z1 = relu(np.matmul(W1,test_x[4]).reshape(-1,1)+b1)
z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)
out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)

In [8]:
print(out)

[[0.01876731]
 [0.60840382]
 [0.37282887]]


In [9]:
print(test_y[4])

[0. 1. 0.]


In [10]:
preds = []
true = []
# print(np.argmax(out))
for i in range(len(test_x)):
    z1 = relu(np.matmul(W1,test_x[i]).reshape(-1,1)+b1)
    z2 = relu(np.matmul(W2,z1).reshape(-1,1)+b2)
    out = softmax(np.matmul(W3,z2).reshape(-1,1) + b3)
    preds.append(np.argmax(out))
    true.append(np.argmax(test_y[i]))

In [11]:
print(preds)

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]


In [12]:
print(true)

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
